<a href="https://colab.research.google.com/github/emyesme/CalcificationDetection/blob/feature-pm/DeepLearning_MC_project_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import copy
import time
import torch
import library
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
from os.path import join
import torch.optim as optim
from torchmetrics import AUROC
import torch.utils.data as data
from torchmetrics import F1Score
import torchvision.datasets as datasets
from matplotlib import pyplot as plt
from torchmetrics.functional import auroc
import torchvision.transforms as transforms
from torchmetrics.functional import precision_recall

In [10]:
RESULTS_DIR = os.path.join('/home',
                            'emily',
                            'Desktop',
                            'CAD',
                            'results')


DATA_DIR = os.path.join('/home',
                        'emily',
                        'Desktop',
                        'CAD',
                        'challenge1')

In [11]:
# hyperparameters
batch_size = 32
learning_rate = 0.001
epochs = 30
momentum = 0.9
lr_step_size = 6   # if < epochs, we are using decaying learning rate
lr_gamma = 0.1
data_augmentation = True
dropout = 0.5
activation = nn.ReLU()

# make visible only one GPU at the time
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # <-- should be the ID of the GPU you want to use

# options
# device = "cuda:0"           # put here "cuda:0" if you want to run on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
monitor_display = True      # whether to display monitored performance plots
display_first_n = 0         # how many samples/batches are displayed
num_workers = 2             # how many workers (=threads) for fetching data
pretrained = False          # whether to test a pretrained model (to be loaded) or train a new one
display_errors = True       # whether to display errors (only in pretrained mode)

In [13]:
import torchvision.transforms.functional as TF
import random

random.seed(1)
# We need to use numpy for rotation transformation
# For sigma and mean we nee to do patch wise


class MyTransform:
    """Rotate by one of the given angles."""

    def __init__(self, times, mode):
        self.times = times
        self.mode = mode

    def __call__(self, x):
        mode = random.choice(self.mode)
        if mode == 0:
          return np.fliplr(x)
        elif mode == 1:
          return np.flipud(x)
        else:   
          times = random.choice(self.times)
          return np.rot90(x, times)
# -90, 0, 90, and 180 degrees rotation

#rotation_transform = MyRotationTransform(angles=[-90, 0, 90, 180])

# DataAugmentation = transforms.RandomApply(
#         [ np.RandomHorizontalFlip(), 
#      transforms.RandomVerticalFlip(),
#      #transforms.RandomRotation(90, fill=(0,)),
#       MyRotationTransform(times=[1,2,3])] , p=0.5)  # fill=(0,) is a workaround for the torchvision bug tracked at https://github.com/pytorch/vision/issues/1759#issuecomment-575307516

class Convert(object):
    def __call__(self, img):
        return torch.unsqueeze(torch.from_numpy(img.astype('float')), 0).float()

transform_train = transforms.Compose(
    [MyTransform(times=[1,2,3], mode=[0,1,2,3])]) 

# should randomly apply a transformation from the list

**Load information**

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()#,
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()#,
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


image_datasets = {x: datasets.ImageFolder(os.path.join(x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
dataset_train.class_total_counts

Train test split

In [ ]:
# train_size = int(0.8 * len(dataset_train))
# validation_size = len(dataset_train) - train_size
# train_dataset, val_dataset = torch.utils.data.random_split(dataset_train, [train_size, validation_size])

In [ ]:
# train_percentage = 0.6
# test_percentage = 0.25

# train_size = int(len(dataset)*train_percentage)
# test_size = int(len(dataset)*test_percentage)

# indices = list(range(len(dataset)))
# np.random.shuffle(indices)
# train_indices, test_indices, val_indices = indices[:train_size], indices[train_size:train_size+test_size], indices[train_size+test_size:]
# train_features = data.SubsetRandomSampler(train_indices)
# val_features = data.SubsetRandomSampler(val_indices)
# test_features = data.SubsetRandomSampler(test_indices)

# dataloader_train = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, sampler=train_features)
# dataloader_val = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, sampler=val_features)
# dataloader_test = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, sampler=test_features)

**Dataloader**

In [ ]:
random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
np.random.seed(1)


dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True) 
dataloader_val = torch.utils.data.DataLoader(dataset_validation, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)  # what is pin memory true

dataloader = {'train': dataloader_train, 'eval': dataloader_val}
dataset_sizes = {'train': len(dataset_train), 'eval': len(dataset_validation)}

### **Model definition**

In [ ]:
import numpy as np
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchsummary import summary

class IncrementalBlock(nn.Module):
    def __init__(self, in_channels):
        super(IncrementalBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.init_weigts()
        
    def init_weigts(self):
        nn.init.xavier_uniform_(self.conv1.weight)
        self.conv1.bias.data.fill_(0.0)
        nn.init.xavier_uniform_(self.conv2.weight)
        self.conv2.bias.data.fill_(0.0)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        return x


class ClassificationBlock(nn.Module):
    def __init__(self, in_features=288):
        super(ClassificationBlock, self).__init__()
        
        self.fc1 = nn.Linear(in_features=in_features, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=256)
        self.out = nn.Linear(in_features=256, out_features=2)
        self.init_weigts()
        
    def init_weigts(self):
        nn.init.xavier_uniform_(self.fc1.weight)
        self.fc1.bias.data.fill_(0.0)
        nn.init.xavier_uniform_(self.fc2.weight)
        self.fc2.bias.data.fill_(0.0)
        nn.init.xavier_uniform_(self.out.weight)
        self.out.bias.data.fill_(0.0)
        
    def forward(self, x):
        x = F.dropout(self.fc1(x))
        x = F.dropout(self.fc2(x))
        x = self.out(x)
        return x


class CNN_final(nn.Module):
    def __init__(self, input_size):
        super(CNN_final, self).__init__()
        
        self.num_ic = int(np.log2(input_size))-1
        incremental_blocks = OrderedDict([])
        for i in range(self.num_ic):
            if i == 0:
                in_channels = 1
            else:
                in_channels = 32
            incremental_blocks.update({f"IC{i}": IncrementalBlock(in_channels)})
        self.conv = nn.ModuleDict({"incremental_block": nn.Sequential(incremental_blocks)})
        self.flatten = nn.Flatten()
        self.classifier = ClassificationBlock()

    def forward(self, x):
        for i in range(self.num_ic):
            x = self.conv.incremental_block[i](x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x


In [ ]:
# define CNN

###
# Haq, I.U., Ali, H., Yu, W.H., Lei, C., Ali, H., Feature fusion and ensemble learningbased CNN model for mammographic image classification, Journal of King Saud University - Computer and
# Information Sciences (2022), doi: https://doi.org/10.1016/j.jksuci.2022.03.023
###

## leakyrelu used instead of elu
## Adam optimizer with a learning rate of 0.0001, β1 of 0.9, and β2 of 0.99, and a batch size of 24

class CD_CNN(nn.Module):
    def __init__(self):
        super(CD_CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1) #kernel_size=7
        # kernel sizes reduced due to patch size
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1) #kernel_size=7
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # stride? padding?
        #https://stackoverflow.com/questions/63971920/why-am-i-getting-calculated-padding-input-size-per-channel-smaller-than-kernel-s
        self.dropout1=nn.Dropout(p=0.2)
        self.bn3 = nn.BatchNorm2d(64)

        #self.dwconv1 = nn.DepthwiseConv2D(in_channels=64, out_channels=128, kernel_size=5, padding=1)
        self.dwconv1 = nn.Conv2d(in_channels=64, out_channels=128, groups= 64, kernel_size=3, padding=1) #kernel_size=5
        #https://discuss.pytorch.org/t/attributeerror-module-torch-nn-has-no-attribute-conv2d/90038
        #https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html        
        self.bn4 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1) #kernel_size=5
        self.bn5 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2=nn.Dropout(p=0.2)
        self.bn6 = nn.BatchNorm2d(128)

        #self.dwconv2 = nn.DepthwiseConv2D(in_channels=128, out_channels=256, kernel_size=3, padding=1)
       
        # third block removed due to patch size
        # self.dwconv2 = nn.Conv2d(in_channels=128, out_channels=256, groups= 128, kernel_size=3, padding=1)
        # self.bn7 = nn.BatchNorm2d(256)
        # self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        # self.bn8 = nn.BatchNorm2d(256)
        # self.conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        # self.bn9 = nn.BatchNorm2d(256)
        # self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        # self.dropout3=nn.Dropout(p=0.2)
        # self.bn10 = nn.BatchNorm2d(256)

        self.fc1 = nn.Linear(128*3*3,512) #128*3*3 is output size of previous stage
        self.fc2 = nn.Linear(512,2) # 2?

        self.out = nn.Sigmoid()

        
        
        self.act = activation

        #self.act = nn.Sigmoid()

        nn.init.xavier_normal_(self.conv1.weight)
        nn.init.xavier_normal_(self.conv2.weight)
        nn.init.xavier_normal_(self.conv3.weight)
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)
       # nn.init.xavier_normal_(self.conv4.weight)
       # nn.init.xavier_normal_(self.conv5.weight)
        nn.init.xavier_normal_(self.dwconv1.weight)
       # nn.init.xavier_normal_(self.dwconv2.weight)


    def forward(self, x):
        
        x = self.act(self.bn1(self.conv1(x)))
        x = self.act(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.bn3(self.dropout1(x))
        # print(x.size())

        x = self.act(self.bn4(self.dwconv1(x)))
        x = self.act(self.bn5(self.conv3(x)))
        x = self.pool2(x)
        x = self.bn6(self.dropout2(x))
        # print(x.size())

        # x = self.act(self.bn7(self.dwconv2(x)))
        # x = self.act(self.bn8(self.conv4(x))) 
        # x = self.act(self.bn9(self.conv5(x))) 
        # x = self.pool3(x)
        # x = self.bn10(self.dropout3(x))
        # print(x.size())

        x = x.view(-1, 128*3*3)
        x = self.act(self.fc1(x))

        # print(x.size())

        x = self.fc2(x)
        # no activation: "remember in this case to not specify the activation in the last layer when you define the net!"
        # consider softmax/logsoftmax
        
        # print(x.size())
        
        x = self.out(x)

        # print(x.size())

        return x

In [ ]:
class DeepNet(nn.Module):
    def __init__(self):
       super(DeepNet, self).__init__()
       self.fc1 = nn.Linear(144, 100)
       self.fc2 = nn.Linear(100, 100)
       self.fc3 = nn.Linear(100, 10)

       nn.init.xavier_normal_(self.fc1.weight)
       nn.init.xavier_normal_(self.fc2.weight)
       nn.init.xavier_normal_(self.fc3.weight)

       self.bn1 = nn.BatchNorm1d(100)
       self.bn2 = nn.BatchNorm1d(100)

       self.act = activation

       self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        
        x = self.act(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.act(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)

        return x

In [ ]:
class CNN1(nn.Module):
  def __init__(self) -> None:
      super(CNN1, self).__init__()

      # in_channels is number of channels (this is because we have a patch of 1 image)
      # out_channels is number of filters or kernels given out by the convolution
      # stride is a parameter (default is 1)
      self.conv1 =  nn.Conv2d(in_channels=1, out_channels=64, kernel_size=2, padding=1)
      self.bn1 = nn.BatchNorm2d(64)
      self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=2, padding=1)
      self.bn2 = nn.BatchNorm2d(64)
      self.pool1 = nn.MaxPool2d(kernel_size=2)
      self.dropout1=nn.Dropout(p=0.2)
      self.bn3 = nn.BatchNorm2d(64)

      self.flatten = nn.Flatten()

      self.fc1 = nn.Linear(3136,512)
      self.fc2 = nn.Linear(512,2)

      nn.init.xavier_normal_(self.fc1.weight)
      nn.init.xavier_normal_(self.fc2.weight)
      nn.init.xavier_normal_(self.conv1.weight)
      nn.init.xavier_normal_(self.conv2.weight)

  def forward(self, x):

      x = self.conv1(x)
      x = self.bn1(x)
      x = self.conv2(x)
      x = self.conv2(x)
      x = self.pool1(x)
      x = self.dropout1(x)
      x = self.bn3(x)
      x = self.flatten(x)
      x = self.fc1(x)
      x = activation(x)
      x = self.fc2(x)

      return x



In [ ]:
from torchsummary import summary

# This part is to check the parameters of my network and the numbers to come

# net = CD_CNN()
# summary(net, (1,12,12))

# dummy_variable = torch.rand(1,1,12,12) #batch size, channel, image size
# net(dummy_variable).size()

**Useful Metrics and Functions**

In [ ]:
def confusion(prediction, truth):
    """ Returns the confusion matrix for the values in the `prediction` and `truth`
    tensors, i.e. the amount of positions where the values of `prediction`
    and `truth` are
    - 1 and 1 (True Positive)
    - 1 and 0 (False Positive)
    - 0 and 0 (True Negative)
    - 0 and 1 (False Negative)
    """

    confusion_vector = prediction / truth
    # Element-wise division of the 2 tensors returns a new tensor which holds a
    # unique value for each case:
    #   1     where prediction and truth are 1 (True Positive)
    #   inf   where prediction is 1 and truth is 0 (False Positive)
    #   nan   where prediction and truth are 0 (True Negative)
    #   0     where prediction is 0 and truth is 1 (False Negative)

    true_positives = torch.sum(confusion_vector == 1).item()
    false_positives = torch.sum(confusion_vector == float('inf')).item()
    true_negatives = torch.sum(torch.isnan(confusion_vector)).item()
    false_negatives = torch.sum(confusion_vector == 0).item()

    return true_positives, false_positives, true_negatives, false_negatives

In [ ]:
def mcc(tp, fp, tn, fn):
  # Formula taken from https://en.wikipedia.org/wiki/Phi_coefficient
  num = (tp*tn) - (fp*fn)
  den = ((tp+fp) * (tp+fn) * (tn+fp) * (tn+fn)) ** 0.5

  if den != 0:
    return num/den
  else:
    return 0

In [ ]:
def save(model, path_to_save: str) -> None:
    torch.save(model.state_dict(), path_to_save)

def load(model, path_to_model: str):
    return model.load_state_dict(torch.load(path_to_model))

**Train and test function**

In [ ]:

# define train function (1 epoch)
# returns average loss and accuracy
def train(dataset, dataloader):

    # switch to train mode
    net.train()

    # reset performance measures
    loss_sum = 0.0
    correct = 0

    # 1 epoch = 1 complete loop over the dataset
    for batch in dataloader:

        # get data from dataloader. This is the thing that I get from the get_item
        inputs, targets = batch

        # move data to device
        inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward pass
        outputs = net(inputs)

        # calculate loss
        loss = criterion(outputs, targets)

        # loss gradient backpropagation
        loss.backward() # I calculate the derivatives backwards

        # net parameters update
        optimizer.step() #I use the gradients to update the weights

        # accumulate loss
        loss_sum += loss.item()

        # accumulate correct outputs (for accuracy calculation)
        outputs_max = torch.argmax(outputs, dim=1) #predicted labels
        targets_max = targets #torch.argmax(targets, dim=1)
        correct += outputs_max.eq(targets_max).sum().float() # this is to find out how many predictions were correct

    # step learning rate scheduler
    scheduler.step() #we just need to keep track of the steps since the LR depends on it

    # return average loss and accuracy
    return loss_sum / len(dataloader), 100. * correct / len(dataset)


In [ ]:


def train_model(model, criterion, optimizer, scheduler, model_name, num_epochs=30, load_trained=False):
    since = time.time()
    f1 = F1Score(num_classes=2).to(device)

    if load_trained:
      checkpoint = torch.load(RESULTS_DIR + model_name)
      model.load_state_dict(checkpoint['model_state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      last_epoch = checkpoint['epoch']+1
      loss = checkpoint['loss']

    else:
      last_epoch=0


    best_model_wts = copy.deepcopy(model.state_dict()) #It keeps track of the parameters of the model in certain state
    best_auc = 0.0
  
    for epoch in range(last_epoch, num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)



        # Each epoch has a training and validation phase
        for phase in ['train', 'eval']:
          
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
    #            running_corrects = 0
            f1_history = list()

            y_true = list()
            y_probs = list()


            tp_total = 0
            fp_total = 0
            tn_total = 0
            fn_total = 0 

            # Iterate over data.
            for inputs, labels in dataloader[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                with torch.no_grad():
                    preds = torch.argmax(outputs, dim=1)
                    probs = F.softmax(outputs, dim=1)[:,1]

                # statistics
                running_loss += loss.item() * inputs.size(0)
    #                running_corrects += torch.sum(preds == labels.data)
                f1_history.append(f1(preds, labels.data).double().cpu().numpy()) 
                y_true.append(labels.data.cpu())
                y_probs.append(probs.cpu())
    #                running_prerec += precision_recall(preds, labels.data)

                tp, fp, tn, fn = confusion(preds, labels.data)
                tp_total += tp
                fp_total += fp
                tn_total += tn
                fn_total += fn
    #                print(tp, fp, tn, fn) 

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_f1 = np.mean(f1_history)
            epoch_auc = auroc(torch.cat(y_probs, dim=0), torch.cat(y_true, dim=0), max_fpr=0.0001).item()
            epoch_mcc = mcc(tp_total, fp_total, tn_total, fn_total)

            if phase == 'train':
              scheduler.step()
              torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': epoch_loss,
                }, RESULTS_DIR+ model_name)  


            print('{} Loss: {:.4f} F1-score: {:.4f}'.format(
                phase, epoch_loss, epoch_f1))
            print('MCC: ', epoch_mcc)
            print('AUC: ', epoch_auc)                        

            # deep copy the model
            if phase == 'eval' and epoch_auc > best_auc:
                best_auc = epoch_auc
                best_model_wts = copy.deepcopy(model.state_dict())

            del f1_history, y_true, y_probs


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val mcc: {:4f}'.format(best_auc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# Fix the patch size
# consider something not histogram-based
#LBPs
#Haar Features

In [ ]:
# define test function
# returns predictions
def test(dataset, dataloader):

    # switch to test mode
    net.eval()  

    # initialize predictions
    predictions = []
    reals = [] #torch.zeros(len(dataset), dtype=torch.int64)
    sample_counter = 0

    # do not accumulate gradients (faster)
    with torch.no_grad():

        # test all batches
        for inputs, labels in dataloader:

            # get data from dataloader [ignore labels/targets as they are not used in test mode]
            inputs = inputs.to(device)
            labels = labels.to(device)
            # forward pass
            outputs = net(inputs)

            # store predictions
            outputs_max = torch.argmax(outputs, dim=1)
            predictions.append( outputs_max)
            reals.append(labels.data)
            sample_counter += 1 #We should look for MCC, AUC or F1-score, p-AUC- Ratio of positive negatives 10 - 4, 10-3

                #90 degrees rotations 1-100, 1-10

                #patch wise output into a set of regions
                #Using threshold
                #output probability map image processing

    return predictions, reals

**Model initialization and Training**

In [ ]:
net = CNN_final(12).to(device) #we need to also send the model to the GPU as well

# create loss function
criterion = nn.CrossEntropyLoss() #most used for classification purposes

# create SGD optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005) #most common optimizer is adam

# create learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=lr_gamma)

# experiment ID
experiment_ID = "%s_%s_%s_bs(%d)lr(%.4f_%d_%.1f)m(%.1f)e(%d)act(%s)xavier(yes)da(%s)do(%.1f)BN" % (type(net).__name__, type(criterion).__name__, type(optimizer).__name__,
                batch_size, learning_rate, lr_step_size, lr_gamma, momentum, epochs, type(activation).__name__, data_augmentation, dropout)

In [ ]:
model = train_model(net, criterion, optimizer, scheduler, '/model_12_5.pt', num_epochs=epochs, load_trained=False)

In [ ]:
save(model, RESULTS_DIR+'/model_try1.pt')

## **Model Testing**

In [ ]:
del dataset_train
del dataset_validation

In [ ]:
import roi_cc_project

dataset_test = roi_cc_project.CvROI([noMC_file_test, MC_file_test], os.path.join(DATA_DIR, 'images'), img_prefix='', img_suffix='', img_channel=None, img_list='', train=False,
                 crossvalid=(1, 1), class_weights=None, class_max_counts=None, class_min_counts=None,
                 preprocessing=None, augmentation=None, verbose=True)
# dataset = roi_cc_project.UnlabeledImageROI(os.path.join(DATA_DIR, 'images', '20586908_6c613a14b80a8591_MG_R_CC_ANON.tif'),os.path.join(DATA_DIR, 'groundtruths', '20586908_6c613a14b80a8591_MG_R_CC_ANON.tif'), (12,12), img_channel=None, preprocessing=None, verbose=True)

In [ ]:
# Add weights to the Cross Entropy Loss


In [ ]:
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True) 

In [ ]:
predictions, reals = test(dataset_test, dataloader_test)
accuracy = 100. * predictions.eq(reals).sum().float() / len(dataset_test)
print ("Accuracy on test set is %.2f" % accuracy)

# **Chaos zone**

In [ ]:
net = CNN1().to(device) #we need to also send the model to the GPU as well

# create loss function
criterion = nn.CrossEntropyLoss() #most used for classification purposes

# create SGD optimizer
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum) #most common optimizer is adam

# create learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_step_size, gamma=lr_gamma)

# experiment ID
experiment_ID = "%s_%s_%s_bs(%d)lr(%.4f_%d_%.1f)m(%.1f)e(%d)act(%s)xavier(yes)da(%s)do(%.1f)BN" % (type(net).__name__, type(criterion).__name__, type(optimizer).__name__,
                batch_size, learning_rate, lr_step_size, lr_gamma, momentum, epochs, type(activation).__name__, data_augmentation, dropout)

In [ ]:
model = train_model(net, criterion, optimizer, scheduler, num_epochs=epochs)

In [ ]:
class CNN2_mod(nn.Module):
  def __init__(self) -> None:
      super(CNN2_mod, self).__init__()

      # in_channels is number of channels (this is because we have a patch of 1 image)
      # out_channels is number of filters or kernels given out by the convolution
      # stride is a parameter (default is 1)
      self.conv1 =  nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
      self.bn1 = nn.BatchNorm2d(64)
      self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
      self.bn2 = nn.BatchNorm2d(64)
      self.pool1 = nn.MaxPool2d(kernel_size=2, padding=1)
      self.dropout1=nn.Dropout(p=0.2)
      self.bn3 = nn.BatchNorm2d(64)

      self.flatten = nn.Flatten()

      self.fc1 = nn.Linear(3136,2)
      self.fc2 = nn.Linear(512,2)


      nn.init.xavier_normal_(self.conv1.weight)
      nn.init.xavier_normal_(self.conv2.weight)

      nn.init.xavier_normal_(self.fc1.weight)
      nn.init.xavier_normal_(self.fc2.weight)

  def forward(self, x):

      x = self.conv1(x)
#      x = self.conv2(x)
      x = self.pool1(x)
      x = self.dropout1(x)
      x = self.flatten(x)
      x = self.fc1(x)
      # x = activation(x)
      # x = self.fc2(x)

      return x



In [ ]:
net = CNN2_mod()
# summary(net, (1,12,12))

dummy_variable = torch.rand(1,1,12,12) #batch size, channel, image size
net(dummy_variable).size()

In [ ]:
net = CNN2_mod().to(device) #we need to also send the model to the GPU as well

# create loss function
criterion = nn.CrossEntropyLoss() #most used for classification purposes

# create SGD optimizer
optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=momentum) #most common optimizer is adam

# create learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_step_size, gamma=0.1)

# experiment ID
experiment_ID = "%s_%s_%s_bs(%d)lr(%.4f_%d_%.1f)m(%.1f)e(%d)act(%s)xavier(yes)da(%s)do(%.1f)BN" % (type(net).__name__, type(criterion).__name__, type(optimizer).__name__,
                batch_size, learning_rate, lr_step_size, lr_gamma, momentum, epochs, type(activation).__name__, data_augmentation, dropout)

In [ ]:
model = train_model(net, criterion, optimizer, scheduler, num_epochs=epochs)

In [ ]:
acc = 0
for i in range(len(predictions)):
  acc += predictions[i].eq(reals[i]).sum().float()
accuracy = 100*acc/len(dataset_test)

In [ ]:
f1_score = 0
f1 = F1Score(num_classes=2).to(device)

for i in range(len(predictions)):  
  f1_score += f1(predictions[i], reals[i]).double()

f1_score2 = 100*f1_score/len(predictions)

In [ ]:
f1_score2

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    # mean = np.array([0.485, 0.456, 0.406])
    # std = np.array([0.229, 0.224, 0.225])
    # inp = std * inp + mean
    # inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
# inputs, classes = next(iter(loader.rois[0]))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out)